# Apriori Algorithm Implementation Assignment

### Objective:
You will implement the **Apriori algorithm** from scratch (i.e., without using any libraries like `mlxtend`) to find frequent itemsets and generate association rules.

### Dataset:
Use the [Online Retail Dataset](https://www.kaggle.com/datasets/vijayuv/onlineretail) from Kaggle. You can filter it for a specific country (e.g., `United Kingdom`) and time range to reduce size if needed.

---

## Step 1: Data Preprocessing

- Load the dataset
- Remove rows with missing values
- Filter out rows where `Quantity <= 0`
- Convert Data into Basket Format

👉 **Implement code below**

In [ ]:
import pandas as pd


file_path = r"C:\Users\OnlineRetail.csv"

df = pd.read_csv(file_path, encoding='latin1')
df.dropna(inplace=True)
df = df[df['Quantity'] > 0]
basket = (df.groupby(['InvoiceNo', 'Description'])['Quantity']
            .sum()
            .unstack()
            .fillna(0)
            .gt(0) 
            .astype(int)) 

print(basket.head())
+

## Step 2: Implement Apriori Algorithm
Step-by-Step Procedure:
1. Generate Frequent 1-Itemsets
Count the frequency (support) of each individual item in the dataset.
Keep only those with support ≥ min_support.
→ Result is L1 (frequent 1-itemsets)
2. Iterative Candidate Generation (k = 2 to n)
While L(k-1) is not empty:
a. Candidate Generation

Generate candidate itemsets Ck of size k from L(k-1) using the Apriori property:
Any (k-itemset) is only frequent if all of its (k−1)-subsets are frequent.
b. Prune Candidates
Eliminate candidates that have any (k−1)-subset not in L(k-1).
c. Count Support
For each transaction, count how many times each candidate in Ck appears.
d. Generate Frequent Itemsets
Form Lk by keeping candidates from Ck that meet the min_support.
Repeat until Lk becomes empty.
Implement the following functions:
1. `get_frequent_itemsets(transactions, min_support)` - Returns frequent itemsets and their support
2. `generate_candidates(prev_frequent_itemsets, k)` - Generates candidate itemsets of length `k`
3. `calculate_support(transactions, candidates)` - Calculates the support count for each candidate

**Write reusable functions** for each part of the algorithm.

In [7]:
from itertools import combinations

def calculate_support(transactions, candidates):
   
    support_count = {candidate: 0 for candidate in candidates}
    for transaction in transactions:
        transaction_set = set(transaction)
        for candidate in candidates:
            if candidate.issubset(transaction_set):
                support_count[candidate] += 1
    return support_count

def generate_candidates(prev_frequent_itemsets, k):
    
    candidates = set()
    prev_frequent_list = list(prev_frequent_itemsets)
    
    for i in range(len(prev_frequent_list)):
        for j in range(i + 1, len(prev_frequent_list)):
            union_set = prev_frequent_list[i] | prev_frequent_list[j]
            if len(union_set) == k:
                # Prune: check all subsets of size k-1 are frequent
                subsets = combinations(union_set, k - 1)
                if all(frozenset(subset) in prev_frequent_itemsets for subset in subsets):
                    candidates.add(union_set)
    return candidates

def get_frequent_itemsets(transactions, min_support):
    
    single_items = set()
    for transaction in transactions:
        for item in transaction:
            single_items.add(frozenset([item]))

    support_data = {}
    current_frequent_itemsets = {
        item for item, count in calculate_support(transactions, single_items).items()
        if count >= min_support
    }

    for item, count in calculate_support(transactions, single_items).items():
        if count >= min_support:
            support_data[item] = count

    all_frequent_itemsets = dict(support_data)  # store results

    k = 2
    while current_frequent_itemsets:
        candidates = generate_candidates(current_frequent_itemsets, k)
        candidate_support = calculate_support(transactions, candidates)
        
        current_frequent_itemsets = {
            item for item, count in candidate_support.items()
            if count >= min_support
        }
        
        for item, count in candidate_support.items():
            if count >= min_support:
                support_data[item] = count
                all_frequent_itemsets[item] = count
        
        k += 1

    return all_frequent_itemsets


In [17]:
transactions = [
    ["bread", "milk"],
    ["bread", "diaper", "beer", "egg"],
    ["milk", "diaper", "beer", "cola"],
    ["bread", "milk", "diaper", "beer"],
    ["bread", "milk", "diaper", "cola"]
]

min_support = 2
frequent_itemsets = get_frequent_itemsets(transactions, min_support)

print("Frequent Itemsets (support ≥ 2):")
for itemset, support in sorted(frequent_itemsets.items(), key=lambda x: (-x[1], x[0])):
    print(f"{set(itemset)}: {support}")


Frequent Itemsets (support ≥ 2):
{'bread'}: 4
{'milk'}: 4
{'diaper'}: 4
{'beer'}: 3
{'beer', 'diaper'}: 3
{'milk', 'bread'}: 3
{'bread', 'diaper'}: 3
{'milk', 'diaper'}: 3
{'cola'}: 2
{'cola', 'diaper'}: 2
{'cola', 'milk'}: 2
{'beer', 'bread'}: 2
{'milk', 'beer'}: 2
{'milk', 'bread', 'diaper'}: 2
{'cola', 'diaper', 'milk'}: 2
{'milk', 'beer', 'diaper'}: 2
{'beer', 'bread', 'diaper'}: 2


## Step 3: Generate Association Rules

- Use frequent itemsets to generate association rules
- For each rule `A => B`, calculate:
  - **Support**
  - **Confidence**
- Only return rules that meet a minimum confidence threshold (e.g., 0.5)

👉 **Implement rule generation function below**

## Step 4: Output and Visualize

- Print top 10 frequent itemsets
- Print top 10 association rules (by confidence or lift)

👉 **Output results below**

In [22]:
from itertools import combinations

def generate_association_rules(frequent_itemsets, min_confidence, transactions_count):
    rules = []
    for itemset, support in frequent_itemsets.items():
        if len(itemset) >= 2:
            for i in range(1, len(itemset)):
                for left in combinations(itemset, i):
                    right = tuple(sorted(set(itemset) - set(left)))
                    left_support = frequent_itemsets.get(tuple(sorted(left)), 0)
                    if left_support > 0:
                        confidence = support / left_support
                        if confidence >= min_confidence:
                            rules.append({
                                'Rule': f"{left} => {right}",
                                'Support': round(support, 2),
                                'Confidence': round(confidence, 2)
                            })
    return rules

frequent_itemsets = {
    ('bread',): 0.8,
    ('milk',): 0.8,
    ('diaper',): 0.8,
    ('beer',): 0.6,
    ('bread', 'milk'): 0.6,
    ('bread', 'diaper'): 0.6,
    ('milk', 'diaper'): 0.6,
    ('diaper', 'beer'): 0.6
}

rules = generate_association_rules(frequent_itemsets, 0.5, len(transactions))

for rule in rules:
    print(rule)


{'Rule': "('bread',) => ('milk',)", 'Support': 0.6, 'Confidence': 0.75}
{'Rule': "('milk',) => ('bread',)", 'Support': 0.6, 'Confidence': 0.75}
{'Rule': "('bread',) => ('diaper',)", 'Support': 0.6, 'Confidence': 0.75}
{'Rule': "('diaper',) => ('bread',)", 'Support': 0.6, 'Confidence': 0.75}
{'Rule': "('milk',) => ('diaper',)", 'Support': 0.6, 'Confidence': 0.75}
{'Rule': "('diaper',) => ('milk',)", 'Support': 0.6, 'Confidence': 0.75}
{'Rule': "('diaper',) => ('beer',)", 'Support': 0.6, 'Confidence': 0.75}
{'Rule': "('beer',) => ('diaper',)", 'Support': 0.6, 'Confidence': 1.0}


In [13]:
import pandas as pd
from itertools import combinations

data = [
    ['milk', 'bread', 'butter'],
    ['bread', 'butter'],
    ['milk', 'bread'],
    ['milk', 'bread', 'butter', 'eggs'],
    ['bread', 'eggs']
]
df = pd.DataFrame(data, columns=['Item1', 'Item2', 'Item3', 'Item4']).fillna('')

transactions = []
for _, row in df.iterrows():
    transaction = [item for item in row if item != '']
    transactions.append(transaction)

def apriori(transactions, min_support=0.4):
    itemsets = {}
    items = set(item for transaction in transactions for item in transaction)
    
    # Size-1 itemsets
    for item in items:
        sup = sum([item in t for t in transactions]) / len(transactions)
        if sup >= min_support:
            itemsets[frozenset([item])] = sup
    
    k = 2
    current_itemsets = list(itemsets.keys())
    
    while current_itemsets:
        candidates = set()
        for i in range(len(current_itemsets)):
            for j in range(i + 1, len(current_itemsets)):
                union_set = current_itemsets[i] | current_itemsets[j]
                if len(union_set) == k:
                    candidates.add(union_set)
        
        valid_itemsets = {}
        for candidate in candidates:
            sup = sum([candidate.issubset(t) for t in transactions]) / len(transactions)
            if sup >= min_support:
                valid_itemsets[candidate] = sup
        
        if not valid_itemsets:
            break
        
        itemsets.update(valid_itemsets)
        current_itemsets = list(valid_itemsets.keys())
        k += 1
    
    return itemsets

frequent_itemsets = apriori(transactions, min_support=0.4)

def generate_rules(frequent_itemsets, min_confidence=0.6):
    rules = []
    for itemset in frequent_itemsets:
        if len(itemset) > 1:
            for i in range(1, len(itemset)):
                for antecedent in combinations(itemset, i):
                    antecedent = frozenset(antecedent)
                    consequent = itemset - antecedent
                    if consequent:
                        conf = frequent_itemsets[itemset] / frequent_itemsets[antecedent]
                        if conf >= min_confidence:
                            rules.append((antecedent, consequent, frequent_itemsets[itemset], conf))
    return rules

rules = generate_rules(frequent_itemsets)

print("Top 10 Frequent Itemsets:")
for items, support in sorted(frequent_itemsets.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"{set(items)}: {support:.3f}")

print("\nTop 10 Association Rules (sorted by confidence):")
for A, B, support, confidence in sorted(rules, key=lambda x: x[3], reverse=True)[:10]:
    print(f"{set(A)} => {set(B)} | Support: {support:.3f}, Confidence: {confidence:.3f}")


Top 10 Frequent Itemsets:
{'bread'}: 1.000
{'milk'}: 0.600
{'butter'}: 0.600
{'milk', 'bread'}: 0.600
{'bread', 'butter'}: 0.600
{'eggs'}: 0.400
{'eggs', 'bread'}: 0.400
{'milk', 'butter'}: 0.400
{'milk', 'bread', 'butter'}: 0.400

Top 10 Association Rules (sorted by confidence):
{'eggs'} => {'bread'} | Support: 0.400, Confidence: 1.000
{'milk'} => {'bread'} | Support: 0.600, Confidence: 1.000
{'butter'} => {'bread'} | Support: 0.600, Confidence: 1.000
{'milk', 'butter'} => {'bread'} | Support: 0.400, Confidence: 1.000
{'milk'} => {'butter'} | Support: 0.400, Confidence: 0.667
{'butter'} => {'milk'} | Support: 0.400, Confidence: 0.667
{'milk'} => {'bread', 'butter'} | Support: 0.400, Confidence: 0.667
{'butter'} => {'milk', 'bread'} | Support: 0.400, Confidence: 0.667
{'milk', 'bread'} => {'butter'} | Support: 0.400, Confidence: 0.667
{'bread', 'butter'} => {'milk'} | Support: 0.400, Confidence: 0.667
